In [1]:
!pip install scipy
!pip install qiskit_aer
!pip install qiskit

In [2]:
import numpy as np
import math
import pandas as pd
from random import randint
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from qiskit.circuit import ParameterVector
from scipy.optimize import minimize
from scipy.stats import entropy
from scipy.linalg import logm
from qiskit_aer.aerprovider import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import *
from qiskit.circuit import ClassicalRegister, QuantumRegister, Parameter, ParameterVector
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import *
from Utilities import *
import heapq

In [4]:
def compute_entropy(statevector, num_qubits):
    """Computes von Neumann entropy of the first qubit."""
    rho = partial_trace(statevector, list(range(1, num_qubits))) # Trace out other qubits
    rho_matrix = rho.data # Convert to matrix form
    eigvals = np.linalg.eigvalsh(rho_matrix) # Compute eigenvalues
    entropy_val = -np.sum(eigvals * np.log2(eigvals + 1e-10)) # Avoid log(0)
    return entropy_val

In [12]:
"""
Function to naively add fully parametrized, maximally entangled layer layers times
Calculates gradients for each layer, removes smallest magnitude RY gates with given pruning rate
"""
def NaiveBuilder(params:list, ansatz:QuantumCircuit, layers:int,
                 circuit:QuantumCircuit, hamiltonian:SparsePauliOp, estimator:Estimator, pruning_rates):
    n = circuit.num_qubits
    results = []
    
    for rate in pruning_rates:
        temp_ansatz = ansatz.copy()
        temp_params = params.copy()
        new_params = ParameterVector(f'new_{rate}', layers * n)
        
        for l in range(layers):
            naive_layer = QuantumCircuit(n)
            
            # Add RY gates with new parameters
            for i in range(n):
                naive_layer.ry(new_params[l * n + i], i)
                temp_params.append(1)
                
            # Add CX gates for entanglement
            for i in range(1, n):
                naive_layer.cx(0, i)
                
            temp_ansatz = temp_ansatz.compose(naive_layer)
            
            # Simulate gradients (randomized for now)
            accumulator = [(np.random.rand(), i) for i in range(len(temp_params) - n, len(temp_params))]
            heapq.heapify(accumulator)
            
            # Prune lowest-magnitude parameters
            bound = min(math.floor(rate * n), max(1, n // 2)) # Keep at least n/2 gates
            remove = [heapq.heappop(accumulator)[1] % n for _ in range(bound)]
            
            for i, idx in enumerate(sorted(remove)):
                del naive_layer.data[idx - i]
                del temp_params[-1]
                
            temp_ansatz = temp_ansatz.compose(naive_layer)
            
        final_circuit = circuit.compose(temp_ansatz)
        results.append((rate, final_circuit.depth(), len(temp_params)))
        
    return results

In [13]:
def visualize_pruning_results(results):
    rates, depths, params = zip(*results)
    plt.figure(figsize=(10, 5))
    
    # Plot circuit depth
    plt.subplot(1,2,1)
    plt.plot(rates, depths, marker='o', label='Circuit Depth', color='b')
    plt.xlabel('Pruning Rate')
    plt.ylabel('Circuit Depth')
    plt.legend()
    
    # Plot parameter count
    plt.subplot(1,2,2)
    plt.plot(rates, params, marker='s', label='Parameter Count', color='r')
    plt.xlabel('Pruning Rate')
    plt.ylabel('Number of Parameters')
    plt.legend()
    
    plt.savefig("pruning_results.png")
    print("Visualization saved as 'pruning_results.png'")

In [14]:
def measure_expressivity(circuit):
    """Compute expressivity metrics: depth and entanglement entropy."""
    if isinstance(circuit, RealAmplitudes):
        circuit = circuit.decompose()
        
    depth = circuit.depth()
    
    # Bind random values to all parameters
    param_dict = {param: np.random.uniform(0, 2*np.pi) for param in circuit.parameters}
    #print(type(circuit))
    #print(circuit.parameters)
    bound_circuit = circuit.assign_parameters(param_dict)
    
    # Compute entanglement entropy for a simple state preparation
    state = Statevector.from_instruction(bound_circuit)
    entropy_val = compute_entropy(state, circuit.num_qubits)
    
    return depth, entropy_val

In [16]:
def experiment_expressivity(qubits=4, layers=3):
    """Compare different ansatz expressivity."""
    ansatz_types = {
        "RealAmplitudes": RealAmplitudes(qubits, reps=layers, entanglement='full'),
        "EfficientSU2": EfficientSU2(qubits, reps=layers, entanglement='full'),
        "Custom": QuantumCircuit(qubits)
    }
    
    # Custom Ansatz: RY Layers with CZ gates
    for _ in range(layers):
        for i in range(qubits):
            ansatz_types["Custom"].ry(np.random.rand(), i)
        for i in range(qubits - 1):
            ansatz_types["Custom"].cz(i, i + 1)
        
    results = []
    for name, circuit in ansatz_types.items():
        depth, ent = measure_expressivity(circuit)
        results.append((name, depth, ent))
        
    return results

In [17]:
def compute_expressivity(ansatz: QuantumCircuit, param_vector: ParameterVector, samples=100) -> float:
    """Quantifies expressivity via fidelity with Haar-random states"""
    expressivities = []
    for _ in range(samples):
        # Generate a Haar-random state
        target_state = random_statevector(2**ansatz.num_qubits).data
        # Bind random parameters to the ansatz
        bound_params = {param: np.random.uniform(0, 2 * np.pi) for param in param_vector}
        bound_circuit = ansatz.assign_parameters(bound_params)
        # Simulate the ansatz output state
        ansatz_state = Statevector(bound_circuit).data
        # Compute fidelity
        fidelity = np.abs(np.vdot(target_state, ansatz_state))**2
        expressivities.append(fidelity)
    
    return np.mean(expressivities) # Higher mean = more expressive

In [18]:
def visualize_expressivity(results):
    """Plot expressivity metrics."""
    names, depths, entropies = zip(*results)
    
    fig, ax1 = plt.subplots(figsize=(10, 5))
    
    ax1.set_xlabel("Ansatz Type")
    ax1.set_ylabel("Circuit Depth", color="b")
    ax1.bar(names, depths, color="b", alpha=0.6, label="Depth")
    
    ax2 = ax1.twinx()
    ax2.set_ylabel("Entanglement Entropy", color="r")
    ax2.plot(names, entropies, marker="o", color="r", label="Entropy")
    
    plt.title("Ansatz Expressivity Comparison")
    plt.savefig("expressivity_results.png")
    print("Visualization saved as 'expressivity_results.png'")

In [19]:
if __name__ == "__main__":
    H = SparsePauliOp.from_list([("ZIZZ", 1),("ZZII", 3),("IZZI", 1),("IIZZ", 1)]) # Toy hamiltonian
    circuit = QuantumCircuit(4)
    ansatz = QuantumCircuit(4)
    pruning_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
    pruning_results = NaiveBuilder(
        params=[1,1,1,1], 
        ansatz=ansatz, 
        layers=3, 
        circuit=circuit, 
        hamiltonian=H, 
        estimator=Estimator(), 
        pruning_rates=pruning_rates
    )
    
    visualize_pruning_results(pruning_results)
    
    expressivity_results = experiment_expressivity()
    visualize_expressivity(expressivity_results)

Visualization saved as 'pruning_results.png'
Visualization saved as 'expressivity_results.png'
